In [1]:
import numpy as np
import pandas as pd
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns

/home/peta/petaLab/bayesian-iroha/.venv/lib/python3.11/site-packages/arviz/__init__.py:39: FutureWarning: 
ArviZ is undergoing a major refactor to improve flexibility and extensibility while maintaining a user-friendly interface.
Some upcoming changes may be backward incompatible.
For details and migration guidance, visit: https://python.arviz.org/en/latest/user_guide/migration_guide.html
  warn(


In [4]:
print(f"PyMC version: {pm.__version__}")

# 再現性の確保
RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)

PyMC version: 5.26.1


In [7]:
# --- 1. Data の生成 (N=20 小規模Data) ---
def generate_weighted_scenario_data(n_per_page=20):
    """
    Top Page 改善、Checkout page 悪化の Trap-data 生成
    """
    pages = ["Top", "Detail", "Contract", "Checkout"]

    scenario = {
        'Top': (2000, 1900),  # 100ms 改善（良）
        'Detail': (1500, 1480),  # 20ms（微良）
        'Contract': (1000, 1020),  # 変化なし(微増)
        'Checkout': (1000, 1050)  # 50ms 悪化　★ここが罠
    }

    data = []
    for page in pages:
        mu_pre, mu_post = scenario[page]

        # Pre (対数正規分布)
        lcp_pre = rng.lognormal(mean=np.log(mu_pre), sigma=0.4, size=n_per_page)
        data.extend([{"page": page, "group": "pre", "lcp": val} for val in lcp_pre])

        # Post (対数正規分布)
        lcp_post = rng.lognormal(mean=np.log(mu_post), sigma=0.4, size=n_per_page)
        data.extend([{"page": page, "group": "Post", "lcp": val} for val in lcp_post])

    return pd.DataFrame(data)

In [9]:
df = generate_weighted_scenario_data(n_per_page=20)

df

,page,group,lcp
0,Top,pre,1359.477556
1,Top,pre,1496.391817
2,Top,pre,4685.792572
3,Top,pre,1439.927131
4,Top,pre,2796.987268
...,...,...,...
155,Checkout,Post,932.544677
156,Checkout,Post,1078.324546
157,Checkout,Post,794.391104
158,Checkout,Post,1559.903153


In [14]:
df.groupby(["page", "group"]).agg({"lcp": "mean"}).unstack()  # 確認

# unstack() は、MultiIndex（重層的なインデックス）を持つ Series や DataFrame のインデックスの最下層を、カラム（列）へと展開（ピボット）する効果があります。
# このコードの場合、groupby(["page", "group"]) によってインデックスが page と group の二段構えになりますが、
# .unstack() を付けることで、group（pre/Post）が横に並び、ページごとの比較がしやすくなります。

lcp             
group            Post          pre
page                              
Checkout  1283.415173   977.067146
Contract  1170.364388  1136.894050
Detail    1536.930299  1533.081715
Top       2224.911345  2123.107021